# Exercise 1: Elastic rotating bar

In [1]:
from ngsolve import *
from netgen.occ import *
import ipywidgets as widgets
from ngsolve.comp import GlobalSpace
from ngsolve.webgui import Draw
from ngsolve.solvers import Newton
from ngsolve.comp import DifferentialSymbol
import numpy as np
# ngsglobals.msg_level=10
import time

In [2]:
def curl(gf: GridFunction)->GridFunction:
    return CoefficientFunction( (0, 0, grad(gf)[1,0]-grad(gf)[0,1]) )

In [3]:
shape = Rectangle(1, 0.1).Face()

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.02))

In [4]:
# density
rho = 1

mass = rho*Integrate(CF(1)*dx, mesh)

# move shape's center of mass to origin
center_of_mass = (1/mass*Integrate(rho*x*dx, mesh), 1/mass*Integrate(rho*y*dx, mesh))

shape = shape.Move((-center_of_mass[0], -center_of_mass[1], 0))

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.02))

# moment_of_inertia = Integrate((x**2 + y**2)*dx, mesh)

# SE(3) translation
r = list(center_of_mass)
r = [0, 0]


In [5]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

## time-stepping

Variables are

* position $\phi = r + R x$, an affine linear function, with constraint $R^T R = I$, i.e. $R$ is orthogonal (and thus, by continuity, a rotation matrix)
* velocity $v = a + b \wedge x$, in body-frame

In [6]:
E, nu = 100, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [7]:
tau = 0.00001
tend = 0.5

# elastic displacement at start
u0 = CF((0, 0))
# elastic displacement velocity at start
v0 = CF((0, (y+100)))  #  10*CF((y, -x))

fes = VectorH1(mesh, order=3)
u,testf = fes.TnT()
aform = InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(testf)))*dx

a = BilinearForm(aform).Assemble()
mstar = BilinearForm(rho*u*testf*dx + tau**2/4*aform).Assemble()
mstarinv = mstar.mat.Inverse(inverse="sparsecholesky")
f = LinearForm(fes).Assemble()

In [8]:
# gfu is displacement?!
gfu = GridFunction(fes)
gfv = GridFunction(fes)
# deformation from displacement
gf_deform = GridFunction(fes)

gfu.Set(u0)
gfv.Set(v0)

gf_graphical = GridFunction(fes)
scene = Draw (gf_graphical, deformation=True, order=3)

# function for debug plot
gfcentrif = GridFunction(fes)
fcentrif = Draw (gfcentrif, order=3)

# pseudo-Newmark (sloppy force calculation)
for j in range(int(tend/tau)):
    # centrifugal force
    f = LinearForm(rho*Cross(curl(gfv), Cross(curl(gfv), CF((x, y, z))))[0:2]*testf*dx).Assemble()
    
    gfu.vec.data += tau/2 * gfv.vec
    gfv.vec.data += tau * mstarinv * (f.vec - a.mat * gfu.vec)
    gfu.vec.data += tau/2 * gfv.vec

    # fit in rigid body translation
    gf_deform.Set(CF((x, y)) + gfu)
    center_of_mass = (1/mass*Integrate(rho*(x + gfu[0])*Norm(Det(Grad(gf_deform)))*dx, mesh), 1/mass*Integrate(rho*(y + gfu[1])*Norm(Det(Grad(gf_deform)))*dx, mesh))
    print(center_of_mass)
    r[0] += center_of_mass[0]
    r[1] += center_of_mass[1]
    # gfu.Set(gfu - CF(center_of_mass))

    gf_graphical.Set(gfu) # CF((r[0], r[1])) + 

    scene.Redraw()
    
    gfcentrif.Set(gfu*Norm(Det(Grad(gfu))))
    fcentrif.Redraw()
    
    time.sleep(0.1)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

(-1.8245660245250964e-12, 0.001000009999208353)
(-1.0926659425349078e-11, 0.0020000399919429587)
(-3.450628739448866e-11, 0.0030000899640437693)
(-7.960982459191718e-11, 0.004000159892652376)
(-1.5305300145210135e-10, 0.005000249746358602)
(-2.6134738213075537e-10, 0.006000359485399651)
(-4.1063087785208104e-10, 0.007000489061910341)
(-6.065982494139863e-10, 0.008000638420222741)
(-8.544363873657016e-10, 0.009000807497212535)
(-1.1587673365059902e-09, 0.010000996222690068)
(-1.5235889050499668e-09, 0.01100120451983228)
(-1.9522285857657176e-09, 0.012001432305653014)
(-2.4472963130843604e-09, 0.013001679491508034)
(-3.010649894669029e-09, 0.014001945983630368)
(-3.6433594100102353e-09, 0.015002231683692828)
(-4.345684195659779e-09, 0.016002536489392448)
(-5.117054890165143e-09, 0.017002860295053625)
(-5.956062217998023e-09, 0.018003202992244335)
(-6.860453075448953e-09, 0.01900356447040154)
(-7.827133002066167e-09, 0.020003944617460703)
(-8.852180528851613e-09, 0.02100434332048485)
(-9.

KeyboardInterrupt: 

In [ ]:
help(mesh.SetDeformation)